In [5]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer 
import string 
import csv
import gensim
from gensim import corpora
import re
import pandas as pd
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
tk = WordPunctTokenizer() 

def tokenizing_nltk(doc):
    doc_tokenized = tk.tokenize(doc)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in doc_tokenized:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))

    filtered_sentence = []  
    nltk_stop_words = set(stopwords.words("english"))
    my_stop_words = {"I", "4", "ha", "c", ")", ").", "|", "3", "-", "(","–", "e", "249"}
    
    for w in nltk_lemmaList:  
        if w not in nltk_stop_words and w not in my_stop_words:  
            filtered_sentence.append(w)

    punctuations="?:!.,;-()|/"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)

    return filtered_sentence

In [7]:
def lda(file):
    survey = open(file, 'r')
    questions = survey.readlines()
    questions = [line.lower() for line in questions] 
    questions = [tokenizing_nltk(doc) for doc in questions] 
    
    dictionary = corpora.Dictionary(questions)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in questions]
    
    Lda = gensim.models.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics = 9, alpha = 0.1, id2word = dictionary, passes = 50)

    top_words_per_topic = []
    for t in range(ldamodel.num_topics):
        top_words_per_topic.extend([(t, ) + x for x in ldamodel.show_topic(t, topn = 3)])

    pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words_" + file.rsplit('.', maxsplit=1)[0] + ".csv")
    
    return ldamodel.print_topics(num_words = 3)

In [8]:
for file in os.listdir():
    if file.endswith(".txt"):
        print('File: ', file)
        print('Topics: ', lda(file))

File:  mathematics.txt
Topics:  [(0, '0.069*"faculty" + 0.069*"department" + 0.069*"interaction"'), (1, '0.012*"comment" + 0.012*"advising" + 0.012*"experience"'), (2, '0.081*"lmu" + 0.081*"department" + 0.043*"graduation"'), (3, '0.053*"mathematics" + 0.053*"feel" + 0.053*"confidence"'), (4, '0.051*"experience" + 0.051*"." + 0.051*"advising"'), (5, '0.089*"course" + 0.089*"unit" + 0.089*"department"'), (6, '0.117*"math" + 0.104*"division" + 0.078*"upper"'), (7, '0.088*"advising" + 0.088*"feel" + 0.046*"transition"'), (8, '0.108*"division" + 0.083*"lower" + 0.083*"course"')]
File:  hhsc.txt
Topics:  [(0, '0.080*"please" + 0.080*"indicate" + 0.080*"school"'), (1, '0.086*"course" + 0.086*"?" + 0.086*"taken"'), (2, '0.091*"advising" + 0.074*"department" + 0.074*"science"'), (3, '0.149*"graduate" + 0.101*"school" + 0.053*"please"'), (4, '0.076*"project" + 0.040*"yes" + 0.040*"lmu"'), (5, '0.102*"lmu" + 0.102*"begin" + 0.010*"situation"'), (6, '0.013*"travel" + 0.013*"attending" + 0.013*"lm